In [ ]:
#import packages
from os import listdir, makedirs
from os.path import isdir
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode, JoinNode        # pypeline engine
from pandas import DataFrame, Series,read_csv

#set output file type for FSL to NIFTI_GZ
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# Set study variables
#studyhome = '/Users/catcamacho/Box/SNAP/BABIES'
studyhome = '/home/camachocm2/Analysis/SNAP'
raw_data = studyhome + '/raw'
output_dir = studyhome + '/processed/preproc'
workflow_dir = studyhome + '/workflows'
subjects_info = read_csv(studyhome + '/misc/rest_subjects_info.csv',index_col=None, dtype={'subject_id':str})
subjects_info['subject_id'] = subjects_info['subject_id'].apply(lambda x: x.zfill(4))
subjects_list = subjects_info['subject_id'].tolist()

template_brain = studyhome + '/templates/6mo_T2w_template_2mm.nii.gz'
template_mask = studyhome + '/templates/6mo_T2w_template_2mm_mask.nii.gz'
template_gmmask = studyhome + '/templates/6mo_T2w_template_2mm_gm.nii.gz'
template_atlas = studyhome + '/rois/language_ROIs.nii.gz'
template_atlas_key = 

proc_cores = 4 # number of cores of processing for the workflows

TR = 2.5 # in seconds

In [ ]:
def extract_cluster_betas(cluster_index_file, sample_betas, min_clust_size, subject_ids):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from pandas import DataFrame, Series
    from numpy import unique, zeros_like
    from nipype.interfaces.fsl.utils import ImageMeants
    from os.path import abspath
    
    subject_ids = sorted(subject_ids)
    sample_data = DataFrame(subject_ids, index=None, columns=['Subject'])
    
    cluster_nifti = load(cluster_index_file[0])
    cluster_data = cluster_nifti.get_data()
    clusters, cluster_sizes = unique(cluster_data, return_counts=True)
    
    final_clusters = clusters[cluster_sizes>=min_clust_size]
    for clust_num in final_clusters[1:]:
        temp = zeros_like(cluster_data)
        temp[cluster_data==clust_num] = 1
        temp_nii = Nifti1Image(temp,cluster_nifti.affine)
        temp_file = 'temp_clust_mask.nii.gz'
        save(temp_nii, temp_file)
        
        eb = ImageMeants()
        eb.inputs.in_file = sample_betas
        eb.inputs.mask = temp_file
        eb.inputs.out_file = 'betas.txt'
        eb.run()
        betas = open('betas.txt').read().splitlines()
        sample_data['clust' + str(clust_num)] = Series(betas, index=sample_data.index)
    
    sample_data.to_csv('extracted_betas.csv')
    extracted_betas_csv = abspath('extracted_betas.csv')
    return(extracted_betas_csv)

def correlation_matrix()

In [2]:
import bct